In [ ]:
from tiny_reader import TinyImgReader
from tiny_reader import import_hdf5
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten
from keras.layers import BatchNormalization, ZeroPadding2D, Dropout
from keras import regularizers
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.utils import plot_model
from keras.utils import np_utils

In [ ]:
x_train, y_train, x_val, y_val, label_codes = import_hdf5('/src/dataset/imagenet/tiny-imagenet-200/training_data.h5')
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

In [ ]:
# show data
img = cv2.cvtColor(x_train[100], cv2.COLOR_RGB2BGR)
plt.imshow(img)
plt.show()

img = cv2.cvtColor(x_val[100], cv2.COLOR_RGB2BGR)
plt.imshow(img)
plt.show()

In [ ]:
# Create model
model = Sequential()

#conv-spatial batch norm - relu #1 
model.add(ZeroPadding2D(padding = (2, 2), input_shape=(64, 64, 3)))
model.add(Conv2D(64, (5, 5), kernel_regularizer=regularizers.l2(1e-7)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))

#conv-spatial batch norm - relu #2
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-7)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#conv-spatial batch norm - relu #3
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(1e-7)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))

#conv-spatial batch norm - relu #4
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(1e-7)))
model.add(BatchNormalization(epsilon=1e-06, axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#conv-spatial batch norm - relu #5
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(1e-7)))
model.add(BatchNormalization(epsilon=1e-06, axis=1))
model.add(Activation('relu'))

#conv-spatial batch norm - relu #6
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(1e-7)))
model.add(BatchNormalization(epsilon=1e-06, axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#conv-spatial batch norm - relu #7
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(512, (3, 3), kernel_regularizer=regularizers.l2(1e-7)))
model.add(BatchNormalization(epsilon=1e-06, axis=1))
model.add(Activation('relu'))

#conv-spatial batch norm - relu #8
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(512, (3, 3), kernel_regularizer=regularizers.l2(1e-7)))
model.add(BatchNormalization(epsilon=1e-06, axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#conv-spatial batch norm - relu #9
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(1024, (3, 3), kernel_regularizer=regularizers.l2(1e-7)))
model.add(BatchNormalization(epsilon=1e-06, axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#Affine-spatial batch norm - relu #10
model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.l2(1e-5)))
model.add(BatchNormalization(epsilon=1e-06))
model.add(Activation('relu'))

#Affine-spatial batch norm - softmax #11
model.add(Dense(label_codes.shape[0], kernel_regularizer=regularizers.l2(1e-6)))
model.add(Activation('softmax'))

In [ ]:
# We add metrics to get more results you want to see
#sgd = SGD(lr=0.05, decay=0, momentum=0.9, nesterov=True)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Data preprocessing
x_train = x_train / 255
y_train = np_utils.to_categorical(y_train, num_classes=label_codes.shape[0])
x_val = x_val / 255
y_val = np_utils.to_categorical(y_val, num_classes=label_codes.shape[0])

In [ ]:
# preparing callback
model_check_point_callback = ModelCheckpoint("weights.{epoch:02d}-{acc:.2f}-{loss:.4f}.hdf5",
                                 monitor="val_loss",
                                 verbose=1,
                                 save_best_only=True,
                                 mode="auto")

tensor_board = TensorBoard(log_dir='/src/tensorboard', batch_size=256)

#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
#                              patience=2, min_lr=1e-5, verbose=1)

In [ ]:
print('Training ------------')
model.fit(x_train, y_train, epochs=100, batch_size=256, validation_data = [x_val, y_val], callbacks = [tensor_board, model_check_point_callback])